In [3]:
# !pip install kaggle


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import kaggle
import zipfile



In [5]:
# Ruta al archivo ZIP
zip_path = "../stores/mlunlp-2024-ps-2.zip"

# Abrimos el ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Leer los archivos directamente desde el ZIP
    train_set_ind = pd.read_csv(zip_ref.open('train_personas.csv'))
    train_set_hog = pd.read_csv(zip_ref.open('train_hogares.csv'))
    test_set_ind = pd.read_csv(zip_ref.open('test_personas.csv'))
    test_set_hog = pd.read_csv(zip_ref.open('test_hogares.csv'))


In [6]:
train_set_hog.columns.values.tolist()

['id',
 'Clase',
 'Dominio',
 'P5000',
 'P5010',
 'P5090',
 'P5100',
 'P5130',
 'P5140',
 'Nper',
 'Npersug',
 'Ingtotug',
 'Ingtotugarr',
 'Ingpcug',
 'Li',
 'Lp',
 'Pobre',
 'Indigente',
 'Npobres',
 'Nindigentes',
 'Fex_c',
 'Depto',
 'Fex_dpto']

In [7]:
def subsidio(x):
    if x['P6585s1'] == 1:
        return 1
    elif x['P6585s2'] == 1:
        return 1
    elif x['P6585s3'] == 1:
        return 1
    elif x['P6585s4'] == 1:
        return 1
    else:
        return 0

In [8]:
def infancia(x):
    if x['P6040'] <= 15:
        return 1
    else:
        return 0

#### Conjunto de entrenamiento - Individuos

In [9]:
# Generamos una dummy para varón y otra para mujer

train_set_ind_2 =  pd.get_dummies(train_set_ind, columns=['P6020'])
train_set_ind_2.rename(columns={'P6020_1':'Varon','P6020_2':'Mujer'}, inplace=True)

In [10]:
# Generamos una dummy respecto si el individuo cobra subsidio o no

train_set_ind_2['subsidio'] = train_set_ind_2.apply(subsidio, axis=1)

In [11]:
# Generamos una dummy identificatoria de infancia (sujeto con 15 años o menos)

train_set_ind_2['infancia'] = train_set_ind_2.apply(infancia, axis=1)

In [12]:
# Creamos variable de ingreso de capital y/o pensiones

train_set_ind_2['ing_capital'] = train_set_ind_2['P7495'].apply(lambda x: 1 if x == 1 else 0)

In [13]:
# Diccionario de agrupaciones para la variable Oficio
agrupaciones = {
    "Ciencias, Ingeniería y Tecnología": [1, 2, 3, 4, 8],
    "Ciencias Biológicas y de la Salud": [5, 6, 7],
    "Ciencias Sociales y Humanidades": [9, 11, 12, 13, 19],
    "Arte, Cultura y Medios": [15, 16, 17, 18, 92],
    "Administración y Gestión": [20, 21, 30],
    "Oficios Administrativos y Comerciales": [31, 32, 33, 34, 38, 39, 40, 41, 42, 43, 44, 45, 49],
    "Servicios Personales y Comunitarios": [14, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
    "Agricultura, Pesca y Explotación de Recursos Naturales": [60, 61, 62, 63, 64, 71, 72, 73],
    "Industria, Manufactura y Construcción": [70, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 93, 94, 95, 96],
    "Transporte y Logística": [35, 36, 37, 97, 98],
    "Otros Oficios y Trabajos No Clasificados": [99]
}

# Función para asignar la nueva categoría
def asignar_categoria(codigo):
    for categoria, codigos in agrupaciones.items():
        if codigo in codigos:
            return categoria
    return "Sin Categoría"  # En caso de que algún código no esté clasificado

# Verificamos si la columna 'oficio' existe en train_set_ind_2
if 'Oficio' in train_set_ind_2.columns:
    # Aplicamos la transformación
    train_set_ind_2['oficio_new'] = train_set_ind_2['Oficio'].apply(asignar_categoria)
else:
    print("La columna 'oficio' no está presente en train_set_ind_2.")


In [14]:
# Generar columnas dummy para 'oficio_new'
train_set_ind_2 = pd.get_dummies(train_set_ind_2, columns=['oficio_new'], prefix='oficio', drop_first=False)

# Verificar las nuevas columnas creadas
print(train_set_ind_2.head())



                         id  Orden  Clase   Dominio  Estrato1  P6040  P6050  \
0  8d0193e328305a042001a35f      2      1  MEDELLIN         2     38      2   
1  8d0193e328305a042001a35f      5      1  MEDELLIN         2      3      3   
2  8d0193e328305a042001a35f      1      1  MEDELLIN         2     41      1   
3  8d0193e328305a042001a35f      4      1  MEDELLIN         2      3      3   
4  8d0193e328305a042001a35f      3      1  MEDELLIN         2     11      3   

   P6090  P6100  P6210  ...  oficio_Arte, Cultura y Medios  \
0    1.0    1.0    3.0  ...                          False   
1    NaN    NaN    2.0  ...                          False   
2    1.0    1.0    5.0  ...                          False   
3    NaN    NaN    2.0  ...                          False   
4    NaN    NaN    4.0  ...                          False   

   oficio_Ciencias Biológicas y de la Salud  \
0                                     False   
1                                     False   
2          

In [15]:
# Seleccionar las columnas dummy que comienzan con 'oficio_'
dummy_columns = [col for col in train_set_ind_2.columns if col.startswith('oficio_')]

# Convertir estas columnas de tipo bool a tipo int (0 y 1)
train_set_ind_2[dummy_columns] = train_set_ind_2[dummy_columns].astype(int)

# Verificar el resultado
print(train_set_ind_2[dummy_columns].head())


   oficio_Administración y Gestión  \
0                                0   
1                                0   
2                                0   
3                                0   
4                                0   

   oficio_Agricultura, Pesca y Explotación de Recursos Naturales  \
0                                                  0               
1                                                  0               
2                                                  0               
3                                                  0               
4                                                  0               

   oficio_Arte, Cultura y Medios  oficio_Ciencias Biológicas y de la Salud  \
0                              0                                         0   
1                              0                                         0   
2                              0                                         0   
3                              0                  

In [16]:
# Nos quedamos con la máxima cantidad de años de educación del jefe de hogar

train_set_head = train_set_ind_2[train_set_ind_2['P6050'] == 1]

# Guardar las dummies del jefe de hogar junto con otras columnas relevantes
columns_to_keep = ['id', 'P6210s1', 'P6800', 'subsidio', 'P6040'] + dummy_columns
train_set_head = train_set_head[columns_to_keep]

# Renombrar las columnas para indicar que pertenecen al jefe de hogar
train_set_head = train_set_head.rename(columns={
    'P6210s1': 'anios_educ_head',
    'P6800': 'hs_work_head',
    'subsidio': 'subsidio_head',
    'P6040': 'edad_head',
    **{col: f"{col}_head" for col in dummy_columns}  # Renombrar dummies para jefe de hogar
})

# Hacer el merge con train_set_ind_2 para agregar las variables del jefe de hogar
train_set_ind_2 = train_set_ind_2.merge(train_set_head, how='left', on='id')

train_set_ind_2

,id,Orden,Clase,Dominio,Estrato1,P6040,P6050,P6090,P6100,P6210,...,"oficio_Arte, Cultura y Medios_head",oficio_Ciencias Biológicas y de la Salud_head,oficio_Ciencias Sociales y Humanidades_head,"oficio_Ciencias, Ingeniería y Tecnología_head","oficio_Industria, Manufactura y Construcción_head",oficio_Oficios Administrativos y Comerciales_head,oficio_Otros Oficios y Trabajos No Clasificados_head,oficio_Servicios Personales y Comunitarios_head,oficio_Sin Categoría_head,oficio_Transporte y Logística_head
0,8d0193e328305a042001a35f,2,1,MEDELLIN,2,38,2,1.0,1.0,3.0,...,0,0,0,0,0,1,0,0,0,0
1,8d0193e328305a042001a35f,5,1,MEDELLIN,2,3,3,NaN,NaN,2.0,...,0,0,0,0,0,1,0,0,0,0
2,8d0193e328305a042001a35f,1,1,MEDELLIN,2,41,1,1.0,1.0,5.0,...,0,0,0,0,0,1,0,0,0,0
3,8d0193e328305a042001a35f,4,1,MEDELLIN,2,3,3,NaN,NaN,2.0,...,0,0,0,0,0,1,0,0,0,0
4,8d0193e328305a042001a35f,3,1,MEDELLIN,2,11,3,NaN,NaN,4.0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543104,5f037d9ddb23a0223d937b30,2,1,BOGOTA,2,24,2,1.0,1.0,5.0,...,0,0,0,0,0,1,0,0,0,0
543105,e575d8c70d587b3be3f45a6f,1,1,BOGOTA,3,36,1,1.0,1.0,6.0,...,0,0,0,0,0,0,0,0,0,0
543106,e575d8c70d587b3be3f45a6f,2,1,BOGOTA,3,41,2,1.0,1.0,6.0,...,0,0,0,0,0,0,0,0,0,0
543107,e575d8c70d587b3be3f45a6f,3,1,BOGOTA,3,8,3,NaN,NaN,3.0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Identificamos las columnas dummy del jefe de hogar 
dummy_columns_head = [col for col in train_set_ind_2.columns if col.startswith('oficio_') and col.endswith('_head')]

# Crear la tabla pivot
train_set_ind_group = pd.pivot_table(
    data=train_set_ind_2,
    values=['Ingtotob', 'Ingtot', 'Varon', 'Mujer', 'subsidio', 'infancia', 'ing_capital',
            'anios_educ_head', 'hs_work_head', 'subsidio_head', 'edad_head'] + dummy_columns_head,
    index=['id', 'Dominio', 'Fex_c', 'Fex_dpto'],
    aggfunc={
        'Ingtotob': 'sum',  # Suma de los ingresos totales observados
        'Ingtot': 'sum',    # Suma de los ingresos totales
        'Varon': 'sum',     # Número total de hombres
        'Mujer': 'sum',     # Número total de mujeres
        'subsidio': 'mean', # Promedio de individuos con subsidio
        'infancia': 'sum',  # Número total de individuos en infancia
        'ing_capital': 'first',  # Primer valor de ingreso por capital (único para jefe)
        'anios_educ_head': 'first',  # Primer valor de educación del jefe
        'hs_work_head': 'first',     # Primer valor de horas trabajadas por el jefe
        'subsidio_head': 'first',    # Primer valor del subsidio del jefe
        'edad_head': 'first',        # Primer valor de edad del jefe
        **{col: 'first' for col in dummy_columns_head}  # Usar 'first' para las dummies del jefe
    }
).reset_index()

# Verificamos el resultado
train_set_ind_group


,id,Dominio,Fex_c,Fex_dpto,Ingtot,Ingtotob,Mujer,Varon,anios_educ_head,edad_head,...,oficio_Ciencias Sociales y Humanidades_head,"oficio_Ciencias, Ingeniería y Tecnología_head","oficio_Industria, Manufactura y Construcción_head",oficio_Oficios Administrativos y Comerciales_head,oficio_Otros Oficios y Trabajos No Clasificados_head,oficio_Servicios Personales y Comunitarios_head,oficio_Sin Categoría_head,oficio_Transporte y Logística_head,subsidio,subsidio_head
0,00004a3f76ed8c33d84a5c70,RESTO URBANO,535.091737,243.971657,1.076429e+06,1.076429e+06,3,2,11.0,31,...,0,0,0,0,0,0,1,0,0.200000,0
1,000088edf78cfaaf1791beba,RESTO URBANO,48.637019,80.145844,1.032500e+06,1.032500e+06,2,1,5.0,64,...,0,0,0,0,0,0,1,0,0.000000,0
2,00009b13480f6770b6a4d1e9,MANIZALES,18.831041,15.894821,1.800000e+06,1.800000e+06,1,2,3.0,40,...,0,0,0,1,0,0,0,0,0.000000,0
3,0000b1899fb63a7bf0b73ccf,RURAL,48.193596,87.435510,6.400000e+05,6.400000e+05,1,1,0.0,59,...,0,0,0,0,0,0,0,0,0.000000,0
4,0000f3ec65ccacadfcc59f32,SANTA MARTA,11.184193,10.067744,1.812492e+06,1.812492e+06,2,1,3.0,79,...,0,0,0,0,0,0,1,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164955,fffdd1840d9683d328b17ac1,RESTO URBANO,143.529561,191.466651,1.607500e+06,1.607500e+06,1,1,8.0,67,...,0,0,0,0,0,0,1,0,0.000000,0
164956,fffe6982a5a945d769ff5893,MEDELLIN,101.208574,94.028086,9.561197e+05,9.561197e+05,2,1,5.0,29,...,0,0,0,0,0,0,0,1,0.333333,1
164957,fffeb473112bf14d6b9da295,RURAL,201.076651,103.248713,7.600000e+05,7.600000e+05,3,1,3.0,20,...,0,0,1,0,0,0,0,0,0.000000,0
164958,ffff32451ed4afe313eb8e8d,IBAGUE,30.887591,23.020429,4.000000e+05,0.000000e+00,2,1,5.0,55,...,0,0,0,1,0,0,0,0,0.000000,0


#### Conjunto de prueba - Individuos

In [18]:
# Genero una dummy para varón y otra para mujer

test_set_ind_2 =  pd.get_dummies(test_set_ind, columns=['P6020'])
test_set_ind_2.rename(columns={'P6020_1':'Varon','P6020_2':'Mujer'}, inplace=True)

In [19]:
# Genero una dummy respecto si el individuo cobra subsidio o no

test_set_ind_2['subsidio'] = test_set_ind_2.apply(subsidio, axis=1)

In [20]:
# Genero una dummy identificatoria de infancia (sujeto con 15 años o menos)

test_set_ind_2['infancia'] = test_set_ind_2.apply(infancia, axis=1)

In [21]:
# Creo variable de ingreso de capital y/o pensiones

test_set_ind_2['ing_capital'] = test_set_ind_2['P7495'].apply(lambda x: 1 if x == 1 else 0)

In [22]:
# Me quedo con la máxima cantidad de años de educación del jefe de hogar

test_set_head = test_set_ind_2[test_set_ind_2['P6050'] == 1]

test_set_head = test_set_head[['id','P6210s1','P6800','subsidio','P6040']]

test_set_head = test_set_head.rename(columns={'P6210s1':'anios_educ_head','P6800':'hs_work_head', 
                                                'subsidio':'subsidio_head', 'P6040':'edad_head'})

test_set_ind_2 = test_set_ind_2.merge(test_set_head, how='left', on='id')

test_set_ind_2

,id,Orden,Clase,Dominio,P6040,P6050,P6090,P6100,P6210,P6210s1,...,Fex_dpto,Varon,Mujer,subsidio,infancia,ing_capital,anios_educ_head,hs_work_head,subsidio_head,edad_head
0,3279230a4917cdf883df34cd,1,1,MEDELLIN,33,1,1.0,1.0,6.0,3.0,...,126.041802,True,False,0,0,0,3.0,50.0,0,33
1,3279230a4917cdf883df34cd,3,1,MEDELLIN,7,3,NaN,NaN,3.0,NaN,...,126.041802,False,True,0,1,0,3.0,50.0,0,33
2,3279230a4917cdf883df34cd,2,1,MEDELLIN,37,2,1.0,1.0,6.0,1.0,...,126.041802,False,True,0,0,0,3.0,50.0,0,33
3,3279230a4917cdf883df34cd,4,1,MEDELLIN,5,3,NaN,NaN,2.0,NaN,...,126.041802,True,False,0,1,0,3.0,50.0,0,33
4,01bd1f72445acc719d19bd25,1,1,MEDELLIN,27,1,1.0,1.0,5.0,11.0,...,64.143748,True,False,0,0,1,11.0,48.0,0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219639,95aca5f4d202450cdfc16c37,1,1,RIOHACHA,74,1,1.0,2.0,6.0,5.0,...,5.192896,True,False,0,0,1,5.0,NaN,0,74
219640,95aca5f4d202450cdfc16c37,2,1,RIOHACHA,74,2,1.0,2.0,6.0,5.0,...,5.192896,False,True,0,0,1,5.0,NaN,0,74
219641,95aca5f4d202450cdfc16c37,3,1,RIOHACHA,19,4,1.0,1.0,6.0,5.0,...,5.192896,False,True,0,0,0,5.0,NaN,0,74
219642,95aca5f4d202450cdfc16c37,4,1,RIOHACHA,18,4,1.0,1.0,6.0,0.0,...,5.192896,False,True,0,0,0,5.0,NaN,0,74


In [23]:
test_set_ind_2.columns.values

array(['id', 'Orden', 'Clase', 'Dominio', 'P6040', 'P6050', 'P6090',
       'P6100', 'P6210', 'P6210s1', 'P6240', 'Oficio', 'P6426', 'P6430',
       'P6510', 'P6545', 'P6580', 'P6585s1', 'P6585s2', 'P6585s3',
       'P6585s4', 'P6590', 'P6600', 'P6610', 'P6620', 'P6630s1',
       'P6630s2', 'P6630s3', 'P6630s4', 'P6630s6', 'P6800', 'P6870',
       'P6920', 'P7040', 'P7045', 'P7050', 'P7090', 'P7110', 'P7120',
       'P7150', 'P7160', 'P7310', 'P7350', 'P7422', 'P7472', 'P7495',
       'P7500s2', 'P7500s3', 'P7505', 'P7510s1', 'P7510s2', 'P7510s3',
       'P7510s5', 'P7510s6', 'P7510s7', 'Pet', 'Oc', 'Des', 'Ina',
       'Fex_c', 'Depto', 'Fex_dpto', 'Varon', 'Mujer', 'subsidio',
       'infancia', 'ing_capital', 'anios_educ_head', 'hs_work_head',
       'subsidio_head', 'edad_head'], dtype=object)

In [24]:
test_set_ind_group = pd.pivot_table(data=test_set_ind_2, values=['Varon', 'Mujer', 'subsidio', 'infancia', 'ing_capital',
                                                                  'anios_educ_head','hs_work_head', 'subsidio_head', 'edad_head'],
                                    index=['id', 'Dominio', 'Fex_c', 'Fex_dpto'],
                                    aggfunc= {'Varon':'sum', 'Mujer':'sum', 'subsidio':'mean', 'infancia':'sum',
                                             'ing_capital':'first', 'anios_educ_head':'first','hs_work_head':'first',
                                             'subsidio_head':'first', 'edad_head': 'first'}).reset_index()

#### Conjunto de entrenamiento - Hogar

In [25]:
def tipo_vivienda(x):
    if x == 1 or x == 2:
        return 'Propia'
    elif x == 3 or x == 4:
        return 'Alquilada'
    elif x == 5:
        return 'Tomada'
    else:
        return 'Otro'

In [26]:
train_set_hog['P5090'] = train_set_hog['P5090'].apply(tipo_vivienda)
train_set_hog

,id,Clase,Dominio,P5000,P5010,P5090,P5100,P5130,P5140,Nper,...,Ingpcug,Li,Lp,Pobre,Indigente,Npobres,Nindigentes,Fex_c,Depto,Fex_dpto
0,8d0193e328305a042001a35f,1,MEDELLIN,4,2,Propia,NaN,400000.0,NaN,5,...,5.816661e+05,122809.534158,289878.247150,0,0,0,0,127.222038,5,146.251790
1,cf7491d7f2cc9c216bd009e7,1,MEDELLIN,5,4,Alquilada,NaN,NaN,250000.0,4,...,1.612500e+05,122809.534158,289878.247150,1,0,4,0,115.866093,5,93.080952
2,d90a57f64d2a84dbacbed2a5,1,MEDELLIN,4,2,Propia,NaN,700000.0,NaN,3,...,2.000000e+06,122809.534158,289878.247150,0,0,0,0,100.123276,5,89.637086
3,db34e387a94a783188ab3a33,1,MEDELLIN,3,1,Propia,NaN,450000.0,NaN,1,...,2.450000e+06,122809.534158,289878.247150,0,0,0,0,108.829812,5,95.584376
4,97bbef785824746ecab09c6f,1,MEDELLIN,4,3,Alquilada,NaN,NaN,500000.0,5,...,2.520000e+05,122809.534158,289878.247150,1,0,5,0,110.412859,5,124.560515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164955,0d168c0d27707687fcbf9b60,1,CALI,3,2,Alquilada,NaN,NaN,460000.0,3,...,2.824977e+05,127336.653858,280234.432107,0,0,0,0,73.088259,76,77.806334
164956,757f32d9b76aaa11eb93b4a6,1,BOGOTA,3,3,Alquilada,NaN,NaN,650000.0,8,...,3.600000e+05,123416.961118,280522.423188,0,0,0,0,413.891448,11,413.891448
164957,ff4ecf438cf2375b8d511645,1,BOGOTA,5,4,Propia,NaN,2000000.0,NaN,10,...,6.934544e+05,123416.961118,280522.423188,0,0,0,0,233.379248,11,233.379248
164958,5f037d9ddb23a0223d937b30,1,BOGOTA,2,1,Alquilada,NaN,NaN,400000.0,4,...,4.799466e+05,123416.961118,280522.423188,0,0,0,0,386.721150,11,386.721150


In [27]:
train_set_hog_2 =  pd.get_dummies(train_set_hog, columns=['P5090'])




In [28]:
train_set_hog_2['hacinamiento'] = train_set_hog_2['P5010']/train_set_hog_2['Nper']
train_set_hog_2

,id,Clase,Dominio,P5000,P5010,P5100,P5130,P5140,Nper,Npersug,...,Npobres,Nindigentes,Fex_c,Depto,Fex_dpto,P5090_Alquilada,P5090_Otro,P5090_Propia,P5090_Tomada,hacinamiento
0,8d0193e328305a042001a35f,1,MEDELLIN,4,2,NaN,400000.0,NaN,5,5,...,0,0,127.222038,5,146.251790,False,False,True,False,0.400000
1,cf7491d7f2cc9c216bd009e7,1,MEDELLIN,5,4,NaN,NaN,250000.0,4,4,...,4,0,115.866093,5,93.080952,True,False,False,False,1.000000
2,d90a57f64d2a84dbacbed2a5,1,MEDELLIN,4,2,NaN,700000.0,NaN,3,3,...,0,0,100.123276,5,89.637086,False,False,True,False,0.666667
3,db34e387a94a783188ab3a33,1,MEDELLIN,3,1,NaN,450000.0,NaN,1,1,...,0,0,108.829812,5,95.584376,False,False,True,False,1.000000
4,97bbef785824746ecab09c6f,1,MEDELLIN,4,3,NaN,NaN,500000.0,5,5,...,5,0,110.412859,5,124.560515,True,False,False,False,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164955,0d168c0d27707687fcbf9b60,1,CALI,3,2,NaN,NaN,460000.0,3,3,...,0,0,73.088259,76,77.806334,True,False,False,False,0.666667
164956,757f32d9b76aaa11eb93b4a6,1,BOGOTA,3,3,NaN,NaN,650000.0,8,8,...,0,0,413.891448,11,413.891448,True,False,False,False,0.375000
164957,ff4ecf438cf2375b8d511645,1,BOGOTA,5,4,NaN,2000000.0,NaN,10,10,...,0,0,233.379248,11,233.379248,False,False,True,False,0.400000
164958,5f037d9ddb23a0223d937b30,1,BOGOTA,2,1,NaN,NaN,400000.0,4,4,...,0,0,386.721150,11,386.721150,True,False,False,False,0.250000


#### Conjunto de prueba - Hogares

In [29]:
test_set_hog['P5090'] = test_set_hog['P5090'].apply(tipo_vivienda)

test_set_hog_2 =  pd.get_dummies(test_set_hog, columns=['P5090'])

test_set_hog_2['hacinamiento'] = test_set_hog_2['P5010']/test_set_hog_2['Nper']
test_set_hog_2

,id,Clase,Dominio,P5000,P5010,P5100,P5130,P5140,Nper,Npersug,Li,Lp,Fex_c,Depto,Fex_dpto,P5090_Alquilada,P5090_Otro,P5090_Propia,P5090_Tomada,hacinamiento
0,2a7ddc2779480d7f19834953,1,SANTA MARTA,4,3,NaN,400000.0,NaN,5,5,121449.452925,275594.029448,7.826095,47,7.528079,False,False,True,False,0.600000
1,a0c2e751e582fd49d564f308,1,SANTA MARTA,4,3,NaN,500000.0,NaN,6,6,121449.452925,275594.029448,15.002708,47,15.327736,True,False,False,False,0.500000
2,57273d19e8464a5ff66a582b,2,RURAL,3,1,NaN,250000.0,NaN,2,2,100763.337626,170886.447382,45.701877,17,50.394748,False,False,True,False,0.500000
3,418d052ff7878940ab938601,1,MEDELLIN,4,3,NaN,600000.0,NaN,5,5,122251.781628,294093.665341,70.318877,5,80.725068,False,False,True,False,0.600000
4,212a37fc17016a3c78f76852,1,MEDELLIN,5,2,1800000.0,2000000.0,NaN,2,2,123664.359813,293293.411227,91.241328,5,95.116936,False,False,True,False,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66163,e7d3c7cc62acd33867b7b7ee,1,BARRANQUILLA,3,2,NaN,NaN,450000.0,4,4,127610.370110,292192.080210,48.989768,8,48.415863,True,False,False,False,0.500000
66164,710db9f02de9c616f846c72e,1,CARTAGENA,3,2,NaN,NaN,100000.0,5,5,122811.614203,288078.106437,39.488634,13,32.499700,True,False,False,False,0.400000
66165,5a0d6b6fad06e4736fb6d95f,1,ARMENIA,3,2,NaN,NaN,250000.0,6,6,121282.133083,278659.892172,10.332009,63,9.719097,True,False,False,False,0.333333
66166,569496964b70185785b57789,1,PEREIRA,2,2,NaN,300000.0,NaN,5,5,121522.856805,287292.899913,16.470949,66,14.496007,True,False,False,False,0.400000


#### Merge

In [30]:
train_set_final = train_set_ind_group.merge(train_set_hog_2, how="inner", on="id")

In [31]:
test_set_final = test_set_ind_group.merge(test_set_hog_2, how="inner", on="id")

In [32]:
# Cleaning adicional

train_set_final['Dominio'] = train_set_final['Dominio_x']
train_set_final = train_set_final.drop(columns=['Dominio_x', 'Dominio_y'])

train_set_final['Fex_c'] = train_set_final['Fex_c_x']
train_set_final = train_set_final.drop(columns=['Fex_c_x', 'Fex_c_y'])

train_set_final['Fex_dpto'] = train_set_final['Fex_dpto_x']
train_set_final = train_set_final.drop(columns=['Fex_dpto_x', 'Fex_dpto_y'])

train_set_final.drop(columns='P5100')

train_set_final['anios_educ_head'] = train_set_final['anios_educ_head'].replace(99, np.nan)

train_set_final['P5130'] = train_set_final['P5130'].replace([98,99], np.nan)

test_set_final['Dominio'] = test_set_final['Dominio_x']
test_set_final = test_set_final.drop(columns=['Dominio_x', 'Dominio_y'])

test_set_final['Fex_c'] = test_set_final['Fex_c_x']
test_set_final = test_set_final.drop(columns=['Fex_c_x', 'Fex_c_y'])

test_set_final['Fex_dpto'] = test_set_final['Fex_dpto_x']
test_set_final = test_set_final.drop(columns=['Fex_dpto_x', 'Fex_dpto_y'])

test_set_final.drop(columns='P5100')

test_set_final['anios_educ_head'] = test_set_final['anios_educ_head'].replace(99, np.nan)

test_set_final['P5130'] = test_set_final['P5130'].replace([98,99], np.nan)

In [33]:
train_set_final['Ingtot']

0         1.076429e+06
1         1.032500e+06
2         1.800000e+06
3         6.400000e+05
4         1.812492e+06
              ...     
164955    1.607500e+06
164956    9.561197e+05
164957    7.600000e+05
164958    4.000000e+05
164959    1.200000e+06
Name: Ingtot, Length: 164960, dtype: float64

In [34]:
# nuevas variables logaritmicas para el training set

variables_log =[ 'Ingtot', 'Ingtotob', 'Ingtotug', 'Ingtotugarr', 'Ingpcug', 'P5130']

for var in variables_log:
    train_set_final[var + "_log"] = np.log(train_set_final[var] + 1)

In [35]:
# Creación de nuevas variables

train_set_final['mayoria_infancia'] = train_set_final['infancia']/train_set_final['Nper']
train_set_final['gen_relativo'] = train_set_final['Mujer']/train_set_final['Nper']

test_set_final['mayoria_infancia'] = test_set_final['infancia']/test_set_final['Nper']
test_set_final['gen_relativo'] = test_set_final['Mujer']/test_set_final['Nper']


In [36]:
train_set_final.columns.values.tolist()

['id',
 'Ingtot',
 'Ingtotob',
 'Mujer',
 'Varon',
 'anios_educ_head',
 'edad_head',
 'hs_work_head',
 'infancia',
 'ing_capital',
 'oficio_Administración y Gestión_head',
 'oficio_Agricultura, Pesca y Explotación de Recursos Naturales_head',
 'oficio_Arte, Cultura y Medios_head',
 'oficio_Ciencias Biológicas y de la Salud_head',
 'oficio_Ciencias Sociales y Humanidades_head',
 'oficio_Ciencias, Ingeniería y Tecnología_head',
 'oficio_Industria, Manufactura y Construcción_head',
 'oficio_Oficios Administrativos y Comerciales_head',
 'oficio_Otros Oficios y Trabajos No Clasificados_head',
 'oficio_Servicios Personales y Comunitarios_head',
 'oficio_Sin Categoría_head',
 'oficio_Transporte y Logística_head',
 'subsidio',
 'subsidio_head',
 'Clase',
 'P5000',
 'P5010',
 'P5100',
 'P5130',
 'P5140',
 'Nper',
 'Npersug',
 'Ingtotug',
 'Ingtotugarr',
 'Ingpcug',
 'Li',
 'Lp',
 'Pobre',
 'Indigente',
 'Npobres',
 'Nindigentes',
 'Depto',
 'P5090_Alquilada',
 'P5090_Otro',
 'P5090_Propia',
 'P

In [37]:
train_set_final.to_csv('../stores/train_set_final.csv')

In [38]:
test_set_final.to_csv('../stores/test_set_final.csv')